In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import probnum as pn

import linpde_gp

In [ ]:
import experiment_utils
from experiment_utils import config

config.experiment_name = "0000_cpu_stationary_1d"
config.target = "jmlr"
config.debug_mode = True

In [ ]:
%matplotlib inline

import matplotlib.axes
import matplotlib.ticker

# plt.rcParams.update(config.tueplots_bundle(rel_width=0.5))
plt.rcParams.update(config.tueplots_bundle())

class BeliefPlotter:
    def __init__(self, pde: linpde_gp.problems.pde.PoissonEquation):
        self._pde = pde

        self._plt_grid = np.linspace(*self._pde.domain, 100)

    @staticmethod
    def _build_cond_events_str(conditioned_on: list[str]) -> str:
        for key in conditioned_on:
            match key:
                case "bc":
                    yield r"u\vert_{\partial \Omega} = g"
                case "pde":
                    yield r"-\alpha \Delta u(x_{\mathrm{PDE}, i}) = f(x_{\mathrm{PDE}, i})"
                case "meas":
                    yield r"u(x_{\mathrm{meas}, i}) = u_{\mathrm{meas}, i}"
                case _:
                    raise ValueError(f"Unknown event '{key}'")

    def _configure_axes(self, ax: matplotlib.axes.Axes):
        ax.xaxis.set_major_locator(matplotlib.ticker.FixedLocator([self._plt_grid[0], self._plt_grid[-1]]))
        ax.xaxis.set_major_formatter(matplotlib.ticker.FixedFormatter([rf"${self._plt_grid[0]:.0f}\, \mathrm{{mm}}$", r"$l_{\mathrm{CPU}}$"]))

        ax.xaxis.set_minor_locator(matplotlib.ticker.AutoMinorLocator())
        ax.xaxis.set_minor_formatter(matplotlib.ticker.FormatStrFormatter(r"$%d\,\mathrm{mm}$"))

    def plot_belief(
        self,
        ax,
        u: pn.randprocs.GaussianProcess,
        conditioned_on: list[str] = [],
        X_pde: np.ndarray | None = None,
        f_X_pde: pn.randvars.RandomVariable | None = None,
        boundary_condition: linpde_gp.problems.pde.DirichletBoundaryCondition | None = None,
        solution: pn.Function | None = None,
        X_meas: np.ndarray | None = None,
        u_X_meas: pn.randvars.RandomVariable | None = None,
    ):
        cond_events_str = ', '.join(BeliefPlotter._build_cond_events_str(conditioned_on))

        u_label = (
            fr"$u \mid {cond_events_str}$"
            if len(conditioned_on) > 0
            else "$u$"
        )
        
        u.plot(
            ax,
            self._plt_grid,
            num_samples=10,
            rng=np.random.default_rng(24),
            color="C0",
            label=u_label,
        )
        
        if solution is not None:
            ax.plot(
                self._plt_grid,
                solution(self._plt_grid),
                color="C1",
                label="$u^*$",
            )
        
        for key in conditioned_on:
            if key == "bc":
                g = pn.randvars.asrandvar(boundary_condition.values)

                ax.errorbar(
                    self._pde.domain.boundary,
                    g.mean,
                    yerr=1.96 * g.std,
                    fmt="+",
                    capsize=2,
                    color="C2",
                    label=r"$g$",
                )
            elif key == "pde":
                for i, x in enumerate(X_pde):
                    ax.axvline(
                        x,
                        color="C3",
                        alpha=0.1,
                        linestyle="--",
                        label=(
                            fr"$(x_{{\mathrm{{PDE}}, 1}}, \dots, "
                            fr"  x_{{\mathrm{{PDE}}, {X_pde.shape[0]}}})$"
                            if i == 0
                            else None
                        )
                    )
            elif key == "meas":
                u_X_meas = pn.asrandvar(u_X_meas)

                ax.errorbar(
                    X_meas,
                    u_X_meas.mean,
                    yerr=1.96 * u_X_meas.std,
                    fmt="+",
                    capsize=2,
                    color="C4",
                    label=fr"$(u_{{\mathrm{{meas}}, 1}}, \dots, u_{{\mathrm{{meas}}, {X_meas.shape[0]}}})$",
                )
        
        self._configure_axes(ax)

        ax.yaxis.set_major_formatter(
            matplotlib.ticker.FormatStrFormatter(r"$%d \, {}^{\circ} \mathrm{C}$")
        )

        ax.legend()

    def plot_pred_belief(
        self,
        ax,
        u: pn.randprocs.GaussianProcess,
        conditioned_on: list[str] = [],
        X_pde: np.ndarray | None = None,
        f_X_pde: pn.randvars.RandomVariable | None = None,
    ):
        cond_events_str = ', '.join(BeliefPlotter._build_cond_events_str(conditioned_on))
        
        u_label = (
            fr"$-\alpha \Delta u \mid {cond_events_str}$"
            if len(cond_events_str) > 0
            else r"$-\alpha \Delta u$"
        )

        self._pde.diffop(u).plot(
            ax,
            self._plt_grid,
            num_samples=10,
            rng=np.random.default_rng(24),
            color="C0",
            label=u_label,
        )
        
        self._pde.rhs.plot(
            ax,
            self._plt_grid,
            color="C1",
            label="$f$",
        )
        
        if "pde" in conditioned_on:
            f_X_pde = pn.randvars.asrandvar(f_X_pde)

            ax.errorbar(
                X_pde,
                f_X_pde.mean,
                yerr=1.96 * f_X_pde.std,
                fmt="+",
                capsize=2,
                c="C3",
                label=f"$(f(x_1), \dots, f(x_{{{X_pde.shape[0]}}}))$",
            )
        
        self._configure_axes(ax)

        ax.yaxis.set_major_formatter(
            matplotlib.ticker.FormatStrFormatter(
                r"$%d \, \frac{\mathrm{W}}{\mathrm{mm}^3}$"
            )
        )

        ax.legend()

## Problem Definition

In [ ]:
# [1] https://en.wikichip.org/wiki/intel/microarchitectures/coffee_lake#Quad-Core
# [2] https://ark.intel.com/content/www/us/en/ark/products/134896/intel-core-i59600k-processor-9m-cache-up-to-4-60-ghz.html
# [3] https://ark.intel.com/content/www/us/en/ark/products/134599/intel-core-i912900k-processor-30m-cache-up-to-5-20-ghz.html
# [4] https://journals.aps.org/pr/pdf/10.1103/PhysRev.134.A1058
# [5]

# CPU Dimensions
w_CPU = 9.19  # mm, [1]
l_CPU = 16.0  # ≈ 16.28 mm, [1]
d_CPU = 0.37  # mm, datasheed linked under [3] ("Supplemental Information / Datasheet")
V_CPU = w_CPU * l_CPU * d_CPU  # mm^3

# Thermal properties
alpha_CPU = 1.56 # W/cm K, [4], TODO: Improve this estimate
alpha_CPU *= 10  # W/mm K

TDP_CPU = 95 # W

In [ ]:
domain = linpde_gp.domains.asdomain([0, l_CPU])

In [ ]:
# Heat Sources
rel_core_positions = np.array([0.2, 0.4, 0.6])
X_cores = rel_core_positions * l_CPU

std_core = 1.2

f_CPU = pn.LambdaFunction(
    lambda x: (TDP_CPU / (w_CPU * d_CPU)) * (1 / np.sqrt(2 * np.pi * std_core)) * np.mean(np.exp(-(1 / (2 * std_core ** 2)) * (x[..., None] - X_cores) ** 2), axis=-1),
    input_shape=(),
)

# Heat Sink
f_cooler = pn.LambdaFunction(
    lambda x: np.full_like(x, TDP_CPU / V_CPU),
    input_shape=(),
)

f = pn.LambdaFunction(
    lambda x: f_CPU(x) - f_cooler(x),
    input_shape=(),
)


In [ ]:
f.plot(plt.gca(), np.linspace(*domain, 100), label=r"$f = f_\mathrm{CPU} + f_\mathrm{cooler}$")
plt.legend()

In [ ]:
pde = linpde_gp.problems.pde.PoissonEquation(
    domain=domain,
    rhs=f,
    alpha=alpha_CPU,
)

plotter = BeliefPlotter(pde)

# Prior

In [ ]:
u_prior = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Constant(input_shape=(), value=50.0),
    cov=5.0 ** 2 * linpde_gp.randprocs.kernels.Matern(
        input_shape=(),
        p=3,
        lengthscale=5.0,
    ),
)

In [ ]:
with plt.rc_context(config.tueplots_bundle(nrows=2, rel_width=0.5)):
    fig, ax = plt.subplots(nrows=2, sharex=True)

    plotter.plot_belief(
        ax=ax[0],
        u=u_prior,
    )

    plotter.plot_pred_belief(
        ax=ax[1],
        u=u_prior,
    )

## Posterior (PDE First)

### Conditioning on the PDE

In [ ]:
X_pde = np.linspace(pde.domain[0] + 0.5, pde.domain[1] - 0.5, 15)
f_X_pde = pde.rhs(X_pde)

In [ ]:
u_cond_pde = u_prior.condition_on_observations(
    X=X_pde,
    Y=np.zeros_like(X_pde),
    L=pde.diffop,
    b=-pn.randvars.asrandvar(f_X_pde),
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_cond_pde,
    conditioned_on=["pde"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

# experiment_utils.savefig("pdefirst_01a_u_cond_pde")

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_pde,
    conditioned_on=["pde"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

# experiment_utils.savefig("pdefirst_01b_Du_cond_pde")

### Conditioning on Exact Dirichlet Boundary Conditions

In [ ]:
g = np.array([50.0, 50.0])

bvp = linpde_gp.problems.pde.PoissonEquationDirichletProblem(pde, boundary_values=g)

In [ ]:
u_cond_pde_bc = u_cond_pde.condition_on_observations(
    X=np.asarray(pde.domain.boundary),
    Y=g,
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_cond_pde_bc,
    conditioned_on=["pde", "bc"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
    boundary_condition=bvp.boundary_conditions[0],
)

experiment_utils.savefig("pdefirst_02a_u_cond_pde_bc")

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_pde_bc,
    conditioned_on=["pde", "bc"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

experiment_utils.savefig("pdefirst_02b_Du_cond_pde_bc")

### Condition on Interior Measurements

In [ ]:
X_meas = rel_core_positions * l_CPU + 0.2
u_X_meas = np.array([60.0, 61.0, 60.3])
u_X_meas_noise = pn.randvars.Normal(
    mean=np.zeros_like(u_X_meas),
    cov=0.3 ** 2 * np.eye(u_X_meas.size),
)

In [ ]:
u_cond_pde_meas = u_cond_pde.condition_on_observations(
    X=X_meas,
    Y=u_X_meas,
    b=u_X_meas_noise,
)

In [ ]:
plotter.plot_belief(
    ax=plt.gca(),
    u=u_cond_pde_meas,
    conditioned_on=["pde", "meas"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
    X_meas=X_meas,
    u_X_meas=u_X_meas + u_X_meas_noise,
)

# experiment_utils.savefig("pdefirst_02a_u_cond_pde_bc")

In [ ]:
plotter.plot_pred_belief(
    ax=plt.gca(),
    u=u_cond_pde_meas,
    conditioned_on=["pde"],
    X_pde=X_pde,
    f_X_pde=f_X_pde,
)

# experiment_utils.savefig("pdefirst_02b_Du_cond_pde_bc")